In [2]:
import pandas as pd
import sys
from data_reader import fes_reader, perf_reader
from tqdm import tqdm
import numpy as np

In [3]:
# конвертация путей файлов в зависимости от системы
def replace_slash(file_path):
    platform = sys.platform
    slash_map = {'win32': '\\',
                'cygwin': '\\',
                'darwin': '/',
                'linux2': '/'}
    if platform not in slash_map.keys(): platform = 'linux2'
    return file_path.replace('\\', slash_map[platform])

In [4]:
perf_path = replace_slash('input_data\\Перфорация.xlsx')
fes_path = replace_slash('input_data\\ФЕС.xlsx')

In [5]:
perf_ints, perf_df = perf_reader(perf_path)
fes_df = fes_reader(fes_path)
SOIL_CUT = 60

In [6]:
fes_df

,well,top,bot,нас-ть,пор-ть,прон-ть,soil
0,10063,1719.199951,1722.799951,26.0,18.799999,266.600006,78.400002
1,10063,1722.799951,1733.800049,NaN,NaN,NaN,NaN
2,10063,1733.800049,1738.800049,26.0,17.299999,184.199997,74.699997
3,10063,1738.800049,1741.599976,NaN,NaN,NaN,NaN
4,10063,1741.599976,1742.799976,26.0,16.500000,122.000000,79.000000
...,...,...,...,...,...,...,...
2583,689,1580.000000,1581.199951,NaN,NaN,NaN,NaN
2584,689,1581.199951,1582.399951,26.0,13.100000,30.000000,74.000000
2585,689,1582.399951,1590.400024,NaN,NaN,NaN,NaN
2586,689,1590.400024,1593.800025,26.0,18.900000,230.000000,82.000000


In [ ]:
def is_perf(top, bot, ints):
    if ints is None:
        return False
    for int_perf in ints:
        if (top <= int_perf['top'] <= bot) or (top <= int_perf['bot'] <= bot):
            return True
        elif (top >= int_perf['top']) and (int_perf['bot'] >= bot):
            return True
    return False


In [ ]:
lost_layers = pd.DataFrame(columns=['well', 'top', 'bot', 'soil', 'is_perf'])
for well in tqdm(fes_df['well'].unique()):
    well_df = fes_df[fes_df['well'] == well][['well', 'top', 'bot', 'soil']]
    well_df.dropna(inplace=True)
    ints = perf_ints.get(well)
    well_df['is_perf'] = well_df.apply(lambda x: is_perf(x['top'], x['bot'], ints), axis=1)
    non_perf = well_df[(well_df['is_perf'] == False) & (well_df['soil'] > SOIL_CUT)]
    lost_layers = lost_layers.append(non_perf, ignore_index=True)

In [ ]:
lost_layers


In [10]:
perf_df


,well,date,тип перфорации,тип перфоратора,число отверстий,top,bot,"плотность перфорации, отв/м",type
710,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,40,1617.000000,1619.800049,14,Да
711,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.800049,1629.000000,19,Да
712,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.800049,1629.000000,19,Да
713,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.800049,1629.000000,19,Да
714,271,1957-08-07,ПЕРВИЧ.ПЕРФ.,Кумулятивная,60,1609.000000,1612.000000,20,Да
...,...,...,...,...,...,...,...,...,...
1361,32488,2011-11-28,ПЕРВИЧ.ПЕРФ.,Кумулятивная,25,1734.000000,1736.000000,12,Да
1362,32488,2013-10-24,ПЕРЕСТРЕЛ,Кумулятивная,40,1734.000000,1736.000000,20,Да
1363,32488,2013-10-26,Перфорация с ОПЗ,NaN,0,1734.000000,1736.000000,0,Да
1364,32488,2017-05-21,Перфорация с ГРП,NaN,0,1734.000000,1736.000000,0,Да


In [7]:
well_df = perf_df[perf_df['well'] == perf_df['well'].unique()[0]]

In [9]:
perf_df.round({'top': 1, 'bot': 1})

,well,date,тип перфорации,тип перфоратора,число отверстий,top,bot,"плотность перфорации, отв/м",type
710,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,40,1617.0,1619.8,14,Да
711,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.8,1629.0,19,Да
712,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.8,1629.0,19,Да
713,271,1953-03-18,ПЕРВИЧ.ПЕРФ.,Кумулятивная,158,1620.8,1629.0,19,Да
714,271,1957-08-07,ПЕРВИЧ.ПЕРФ.,Кумулятивная,60,1609.0,1612.0,20,Да
...,...,...,...,...,...,...,...,...,...
1361,32488,2011-11-28,ПЕРВИЧ.ПЕРФ.,Кумулятивная,25,1734.0,1736.0,12,Да
1362,32488,2013-10-24,ПЕРЕСТРЕЛ,Кумулятивная,40,1734.0,1736.0,20,Да
1363,32488,2013-10-26,Перфорация с ОПЗ,NaN,0,1734.0,1736.0,0,Да
1364,32488,2017-05-21,Перфорация с ГРП,NaN,0,1734.0,1736.0,0,Да
